In [12]:
import pandas as pd
import numpy as np


# for Modelling
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, GRU, LSTM, Dense

from tensorflow.keras.models import load_model

# for Evaluation
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_error, r2_score


In [2]:
df= pd.read_csv('ax_df_all.csv')
df

,Time id,Material,Material_Id,Quantity,Total_Value
0,9,E11-200-001,34342,13,1048203.00
1,10,E11-200-001,34342,28,2257668.00
2,11,E11-200-001,34342,38,2978019.06
3,12,E11-200-001,34342,42,3275484.94
4,13,E11-200-001,34342,32,2519632.00
...,...,...,...,...,...
3124,34,C04-200123,146,212,108739.70
3125,35,C04-200123,146,252,128604.24
3126,36,C04-200123,146,254,128352.64
3127,37,C04-200123,146,238,119471.66


In [3]:
# Define a dictionary to map Time_id to dates
time_id_to_date = {
    1: "2021-01-01", 2: "2021-02-01", 3: "2021-03-01", 4: "2021-04-01", 5: "2021-05-01",
    6: "2021-06-01", 7: "2021-07-01", 8: "2021-08-01", 9: "2021-09-01", 10: "2021-10-01",
    11: "2021-11-01", 12: "2021-12-01", 13: "2022-01-01", 14: "2022-02-01", 15: "2022-03-01",
    16: "2022-04-01", 17: "2022-05-01", 18: "2022-06-01", 19: "2022-07-01", 20: "2022-08-01",
    21: "2022-09-01", 22: "2022-10-01", 23: "2022-11-01", 24: "2022-12-01", 25: "2023-01-01",
    26: "2023-02-01", 27: "2023-03-01", 28: "2023-04-01", 29: "2023-05-01", 30: "2023-06-01",
    31: "2023-07-01", 32: "2023-08-01", 33: "2023-09-01", 34: "2023-10-01", 35: "2023-11-01",
    36: "2023-12-01", 37: "2024-01-01", 38: "2024-02-01", 39: "2024-02-01"
}

# Change Time_id column to dates using the dictionary
df['Time id'] = df['Time id'].map(time_id_to_date)

# Optionally, convert the Time_id column to datetime type
df['Time id'] = pd.to_datetime(df['Time id'])

In [4]:
data = df.copy()

In [7]:
data['Material'].unique()

array(['E11-200-001', 'C87-200433', 'S02-060380', 'S02-060352',
       'S01-070021', 'C87-200349', 'C87-400114', 'S01-110220',
       'S02-010175', 'S03-030109', 'S02-060363', 'E58-200-001',
       'S02-060362', 'C87-200560', 'S02-060446', 'E51-200-001',
       'S10-130517', 'SA-A46-07-0100E', 'S02-060445', 'C87-300025',
       'S10-130761', 'C72-100172', 'S25-050054', 'S02-060313',
       'S01-090022', 'S02-060325', 'S06-380001', 'E46-200-001',
       'S10-130309', 'S25-010196', 'S26-090085', 'S10-130652',
       'C04-400146', 'S02-060315', 'S26-090134', 'C87-000231',
       'C87-200463', 'C87-300008', 'C37-300179', 'S06-080061',
       'C90-200051', 'S10-130277', 'C72-000163', 'S12-060213',
       'C60-700112', 'S02-060364', 'S25-010204-01', 'C06-600290',
       'S21-001-320', 'S06-020222', 'S02-060354', 'C37-200275',
       'S12-060139', 'S03-070005', 'C87-400124', 'C87-200403',
       'C04-400148', 'S27-010112', 'S07-471173', 'SA-A07-07-0109',
       'C04-400111', 'S06-080063', 'S0

In [8]:
materials = ['C87-200433', 'S02-060380', 'S02-060352',
       'S01-070021', 'C87-200349', 'C87-400114', 'S01-110220',
       'S02-010175', 'S03-030109', 'S02-060363', 'E58-200-001',
       'S02-060362', 'C87-200560', 'S02-060446', 'E51-200-001',
       'S10-130517', 'SA-A46-07-0100E', 'S02-060445', 'C87-300025',
       'S10-130761', 'C72-100172', 'S25-050054', 'S02-060313',
       'S01-090022', 'S02-060325', 'S06-380001', 'E46-200-001',
       'S10-130309', 'S25-010196', 'S26-090085', 'S10-130652',
       'C04-400146', 'S02-060315', 'S26-090134', 'C87-000231',
       'C87-200463', 'C87-300008', 'C37-300179', 'S06-080061',
       'C90-200051', 'S10-130277', 'C72-000163', 'S12-060213',
       'C60-700112', 'S02-060364', 'S25-010204-01', 'C06-600290',
       'S21-001-320', 'S06-020222', 'S02-060354', 'C37-200275',
       'S12-060139', 'S03-070005', 'C87-400124', 'C87-200403',
       'C04-400148', 'S27-010112', 'S07-471173', 'SA-A07-07-0109',
       'C04-400111', 'S06-080063', 'S02-020065', 'C37-200258',
       'S25-050051', 'S06-070046', 'S06-040109', 'H17-200228',
       'C70-300037', 'C29-300110', 'S10-130616', 'S10-130615',
       'S06-360116', 'C65-200280', 'S23-170045', 'AC61-000002',
       'C37-300152', 'C70-300048', 'C37-500421', 'S10-130537',
       'C04-400202', 'C04-400200', 'C07-000704', 'S02-060395',
       'C97-100180', 'C87-100152', 'C65-200279', 'S10-130625',
       'C04-200123']

# Simple RNN, GRU, LSTM

For this analyis Qunatity column is taken into consideration and unique Material is taken for the forecasting since the data we have is from 2021-09-01 to 2024-02-01 so for the forecasting the train set that is taken is from 2021-09-01 to 2023-10-01 and for test 2023-10-01 to 2024-02-01. Then MinMax Normalizatin technique is used to scale the data.

In [15]:
# Define the evaluate_model function
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mae, r2

# Convert 'Time id' to datetime
data['Time id'] = pd.to_datetime(data['Time id'])

# Define train and test periods
train_start_date = '2021-09-01'
train_end_date = '2023-10-01'
test_start_date = '2023-11-01'
test_end_date = '2024-02-01'

# Define a function to build and train RNN models
def build_and_train_model(model_type, X_train, y_train):
    model = Sequential()
    if model_type == 'SimpleRNN':
        model.add(SimpleRNN(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'GRU':
        model.add(GRU(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    elif model_type == 'LSTM':
        model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    return model

# Iterate over each material
for material in data['Material'].unique():
    print(f"Evaluating RNN models for Material: {material}")
    
    # Filter data for the current material
    material_data = data[data['Material'] == material]

    # Split data into train and test sets
    train = material_data[(material_data['Time id'] >= train_start_date) & (material_data['Time id'] <= train_end_date)]
    test = material_data[(material_data['Time id'] >= test_start_date) & (material_data['Time id'] <= test_end_date)]

    # Scale the data
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train[['Quantity']])
    test_scaled = scaler.transform(test[['Quantity']])

    # Reshape input to be [samples, time steps, features]
    X_train = np.reshape(train_scaled, (train_scaled.shape[0], 1, 1))
    X_test = np.reshape(test_scaled, (test_scaled.shape[0], 1, 1))
    y_train = train_scaled[:, 0]  # Target variable is Quantity
    y_test = test_scaled[:, 0]

    # Build and train Simple RNN model
    simple_rnn_model = build_and_train_model('SimpleRNN', X_train, y_train)

    # Build and train GRU model
    gru_model = build_and_train_model('GRU', X_train, y_train)

    # Build and train LSTM model
    lstm_model = build_and_train_model('LSTM', X_train, y_train)

    # Evaluate models on test set
    simple_rnn_mae, simple_rnn_r2 = evaluate_model(simple_rnn_model, X_test, y_test)
    gru_mae, gru_r2 = evaluate_model(gru_model, X_test, y_test)
    lstm_mae, lstm_r2 = evaluate_model(lstm_model, X_test, y_test)

    print("Simple RNN MAE:", simple_rnn_mae, " R2 Score:", simple_rnn_r2)
    print("GRU MAE:", gru_mae, " R2 Score:", gru_r2)
    print("LSTM MAE:", lstm_mae, " R2 Score:", lstm_r2)
    print("-" * 50)


Evaluating RNN models for Material: E11-200-001


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 348ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step
Simple RNN MAE: 0.19741327166557307  R2 Score: 0.49514318928241696
GRU MAE: 0.20740855336189265  R2 Score: 0.44686208689689
LSTM MAE: 0.20327178984880442  R2 Score: 0.46869936013120606
--------------------------------------------------
Evaluating RNN models for Material: C87-200433


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step
Simple RNN MAE: 0.03481390429217192  R2 Score: 0.9051527426937985
GRU MAE: 0.08581128787426721  R2 Score: 0.3891628862251073
LSTM MAE: 0.10639908359873863  R2 Score: 0.042562480488585575
--------------------------------------------------
Evaluating RNN models for Material: S02-060380


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 771ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step
Simple RNN MAE: 0.0123690466300167  R2 Score: 0.9982765798792055
GRU MAE: 0.14217389439200534  R2 Score: 0.7656579667328998
LSTM MAE: 0.19704647798990382  R2 Score: 0.5510706107187666
--------------------------------------------------
Evaluating RNN models for Material: S02-060352


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


KeyboardInterrupt: 

Since After the evaluation of all the Materials Now We are forecasting for the next four month using the best mmodel that we that got after the analysis. 

So Simple RNN was giving best accuracy for the r2 and mae metric so we have consider Simple RNN 

# Forecast for the Next Four Month

In [23]:
# Convert 'Time id' to datetime
data['Time id'] = pd.to_datetime(data['Time id'])

# Define train period
train_start_date = '2021-09-01'
train_end_date = '2024-02-01'

# Define forecast period
forecast_start_date = '2024-03-01'
forecast_end_date = '2024-06-01'

# Filter data for the train period
train_data = data[(data['Time id'] >= train_start_date) & (data['Time id'] <= train_end_date)]

# Iterate over each material
for material in train_data['Material'].unique():
    print(f"Forecasting for Material: {material}")
    
    # Filter data for the current material
    material_data = train_data[train_data['Material'] == material]

    # Scale the data
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(material_data[['Quantity']])

    # Reshape input to be [samples, time steps, features]
    X_train = np.reshape(train_scaled, (train_scaled.shape[0], 1, 1))
    y_train = train_scaled[:, 0]  # Target variable is Quantity

    # Build and train Simple RNN model
    model = Sequential()
    model.add(SimpleRNN(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

    # Forecast for the next four months
    forecast_dates = pd.date_range(start=forecast_start_date, end=forecast_end_date, freq='MS')
    forecast_values = []

    # Use the trained model to forecast
    for date in forecast_dates:
        # Reshape the date for prediction
        X_forecast = np.array([[date.month / 12]])  # Normalize month to [0, 1]
        X_forecast = np.reshape(X_forecast, (1, 1, 1))

        # Predict using the model
        forecast_value_scaled = model.predict(X_forecast)
        forecast_value = scaler.inverse_transform(forecast_value_scaled)
        forecast_values.append(forecast_value[0, 0])

    # Create DataFrame for the forecasted values
    forecast_df = pd.DataFrame({'Time id': forecast_dates, 'Material': material, 'Forecasted Quantity': forecast_values})
    print(forecast_df)
    print("-" * 50)


Forecasting for Material: E11-200-001


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
     Time id     Material  Forecasted Quantity
0 2024-03-01  E11-200-001            32.869804
1 2024-04-01  E11-200-001            36.949749
2 2024-05-01  E11-200-001            41.010944
3 2024-06-01  E11-200-001            45.049747
--------------------------------------------------
Forecasting for Material: C87-200433


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
     Time id    Material  Forecasted Quantity
0 2024-03-01  C87-200433            12.553573
1 2024-04-01  C87-200433            13.974027
2 2024-05-01  C87-200433            15.387392
3 2024-06-01  C87-200433            16.792040
--------------------------------------------------
Forecasting for Material: S02-060380


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
     Time id    Material  Forecasted Quantity
0 2024-03-01  C65-200279            43.320396
1 2024-04-01  C65-200279            46.612606
2 2024-05-01  C65-200279            49.888779
3 2024-06-01  C65-200279            53.145199
--------------------------------------------------
Forecasting for Material: S10-130625


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
     Time id    Material  Forecasted Quantity
0 2024-03-01  S10-130625             9.309720
1 2024-04-01  S10-130625            10.538587
2 2024-05-01  S10-130625            11.761777
3 2024-06-01  S10-130625            12.978070
--------------------------------------------------
Forecasting for Material: C04-200123


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
     Time id    Material  Forecasted Quantity
0 2024-03-01  C04-200123           213.835175
1 2024-04-01  C04-200123           222.788773
2 2024-05-01  C04-200123           231.692413
3 2024-06-01  C04-200123           240.536606
--------------------------------------------------


In [25]:
import pandas as pd

# Create an empty DataFrame to store all forecasted quantities
all_forecasts_df = pd.DataFrame(columns=['Material', 'Time id', 'Forecasted Quantity'])

# Iterate over each material
for material in train_data['Material'].unique():
    print(f"Forecasting for Material: {material}")
    
    # Filter data for the current material
    material_data = train_data[train_data['Material'] == material]

    # Scale the data
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(material_data[['Quantity']])

    # Reshape input to be [samples, time steps, features]
    X_train = np.reshape(train_scaled, (train_scaled.shape[0], 1, 1))
    y_train = train_scaled[:, 0]  # Target variable is Quantity

    # Build and train Simple RNN model
    model = Sequential()
    model.add(SimpleRNN(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

    # Forecast for the next four months
    forecast_dates = pd.date_range(start=forecast_start_date, end=forecast_end_date, freq='MS')
    forecast_values = []

    # Use the trained model to forecast
    for date in forecast_dates:
        # Reshape the date for prediction
        X_forecast = np.array([[date.month / 12]])  # Normalize month to [0, 1]
        X_forecast = np.reshape(X_forecast, (1, 1, 1))

        # Predict using the model
        forecast_value_scaled = model.predict(X_forecast)
        forecast_value = scaler.inverse_transform(forecast_value_scaled)
        forecast_values.append(forecast_value[0, 0])

    # Create DataFrame for the forecasted values
    forecast_df = pd.DataFrame({'Time id': forecast_dates, 'Material': material, 'Forecasted Quantity': forecast_values})
    
    # Append the forecast DataFrame to the master DataFrame
    all_forecasts_df = pd.concat([all_forecasts_df, forecast_df], ignore_index=True)

# Define the file path where you want to save the CSV
file_path = 'forecasted_quantities_all_materials.csv'

# Save the master DataFrame to CSV
all_forecasts_df.to_csv(file_path, index=False)

print(f"Forecasted quantities for all materials saved to '{file_path}' successfully.")



Forecasting for Material: E11-200-001


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Forecasting for Material: C87-200433


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Forecasting for Material: S02-060380


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 345ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Forecasting for Material: S10-130625


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Forecasting for Material: C04-200123


C:\Users\sohan\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Forecasted quantities for all materials saved to 'forecasted_quantities_all_materials.csv' successfully.


Here i have taken three columns Material, Time id and forecated Value then stored it in everytime the fucntion run and forecast for the next month.

The Forecasted values is taken The forecasted Values is saved into the Local Machine as the Csv file.